In [1]:
!pip install face_recognition

In [2]:
!pip install opencv-python


In [6]:
import face_recognition
import cv2
import numpy as np
import csv
import os
from datetime import datetime

In [7]:
video_capture = cv2.VideoCapture(0)

In [8]:
# Load known faces
Bill_image = face_recognition.load_image_file("photos/Bill.jpg")
Bill_encoding = face_recognition.face_encodings(Bill_image)[0]

Elon_image = face_recognition.load_image_file("photos/Elon.jpg")
Elon_encoding = face_recognition.face_encodings(Elon_image)[0]

Mark_image = face_recognition.load_image_file("photos/Mark.jpg")
Mark_encoding = face_recognition.face_encodings(Mark_image)[0]

Steve_image = face_recognition.load_image_file("photos/Steve.jpg")
Steve_encoding = face_recognition.face_encodings(Steve_image)[0]

Dewmini_image = face_recognition.load_image_file("photos/Dewmini.jpg")
Dewmini_encoding = face_recognition.face_encodings(Dewmini_image)[0]

known_face_encoding = [Bill_encoding, Elon_encoding, Mark_encoding, Steve_encoding, Dewmini_encoding]
known_faces_names = ["Bill", "Elon", "Mark", "Steve", "Dewmini"]
students = known_faces_names.copy()


In [9]:
# Simple liveness detection variables
previous_frame = None
liveness_threshold = 1000  # Minimum motion required
liveness_verified = {}
motion_frames_required = 30  # Frames of motion needed
recognition_delay = 60  # Frames to wait before allowing attendance

# CSV setup
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
f = open(current_date + '.csv', 'w+', newline='')
lnwriter = csv.writer(f)

print("Instructions:")
print("- Move your head slightly or blink naturally")
print("- Stay still for too long = not live")
print("- Green box = Live person, Red = Static/Fake")
print("- Press 'q' to quit")

frame_count = 0


Instructions:
- Move your head slightly or blink naturally
- Stay still for too long = not live
- Green box = Live person, Red = Static/Fake
- Press 'q' to quit


In [10]:
while True:
    ret, frame = video_capture.read()
    
    if not ret:
        print("Failed to capture frame from camera")
        break
    
    frame_count += 1
    
    # Convert to grayscale for motion detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    # Face recognition
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1].astype(np.uint8))
    
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    
    # Motion detection
    motion_detected = False
    if previous_frame is not None:
        # Calculate frame difference
        frame_delta = cv2.absdiff(previous_frame, gray)
        thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        
        # Calculate motion amount
        motion_amount = cv2.countNonZero(thresh)
        motion_detected = motion_amount > liveness_threshold
        
        # Show motion visualization (optional)
        cv2.imshow("Motion Detection", thresh)
    
    previous_frame = gray.copy()
    
    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encoding, face_encoding)
        face_distance = face_recognition.face_distance(known_face_encoding, face_encoding)
        best_match_index = np.argmin(face_distance)
        
        if matches[best_match_index]:
            name = known_faces_names[best_match_index]
        else:
            name = "Unknown"
            
        face_names.append(name)
        
        # Initialize liveness tracking
        if name not in liveness_verified and name != "Unknown":
            liveness_verified[name] = {
                "motion_frames": 0,
                "total_frames": 0,
                "is_live": False,
                "first_seen": frame_count
            }
        
        # Update liveness tracking
        if name in liveness_verified:
            liveness_verified[name]["total_frames"] += 1
            if motion_detected:
                liveness_verified[name]["motion_frames"] += 1
            
            # Calculate liveness percentage
            motion_ratio = liveness_verified[name]["motion_frames"] / max(liveness_verified[name]["total_frames"], 1)
            
            # Verify liveness (need motion in at least 20% of frames and minimum frames)
            if (liveness_verified[name]["total_frames"] >= motion_frames_required and 
                motion_ratio > 0.2):
                liveness_verified[name]["is_live"] = True
    
    # Display faces with liveness status
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        # Determine status and color
        if name == "Unknown":
            color = (0, 0, 255)  # Red
            status = "Unknown"
            is_live = False
        elif name in liveness_verified:
            is_live = liveness_verified[name]["is_live"]
            motion_ratio = liveness_verified[name]["motion_frames"] / max(liveness_verified[name]["total_frames"], 1)
            frames_seen = liveness_verified[name]["total_frames"]
            
            if is_live:
                color = (0, 255, 0)  # Green
                status = "LIVE"
            else:
                color = (0, 165, 255)  # Orange
                if frames_seen < motion_frames_required:
                    status = f"Analyzing... {frames_seen}/{motion_frames_required}"
                else:
                    status = f"Move more! {motion_ratio:.1%}"
        else:
            color = (0, 0, 255)  # Red
            status = "Not recognized"
            is_live = False
        
        # Mark attendance only for live faces
        if (name in students and is_live and 
            frame_count - liveness_verified[name]["first_seen"] > recognition_delay):
            
            students.remove(name)
            print(f"Attendance marked for: {name} (Live person verified)")
            current_time = datetime.now().strftime("%H:%M:%S")
            lnwriter.writerow([name, current_time, "Live"])
            f.flush()
            
            if len(students) == 0:
                print("All students marked present! Closing camera...")
                break
        
        # Draw face rectangle and labels
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.rectangle(frame, (left, bottom - 70), (right, bottom), color, cv2.FILLED)
        
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 45), font, 0.6, (255, 255, 255), 1)
        cv2.putText(frame, status, (left + 6, bottom - 25), font, 0.4, (255, 255, 255), 1)
        
        # Show motion info for recognized faces
        if name in liveness_verified:
            motion_info = f"Motion: {liveness_verified[name]['motion_frames']}/{liveness_verified[name]['total_frames']}"
            cv2.putText(frame, motion_info, (left + 6, bottom - 5), font, 0.3, (255, 255, 255), 1)
    
    # Display overall motion status
    motion_text = "MOTION DETECTED" if motion_detected else "NO MOTION"
    motion_color = (0, 255, 0) if motion_detected else (0, 0, 255)
    cv2.putText(frame, motion_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, motion_color, 2)
    
    # Display remaining students
    cv2.putText(frame, f"Remaining: {len(students)}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    cv2.imshow("Liveness Detection + Attendance", frame)
    
    # Break conditions
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    
    # Auto-close if all students marked
    if len(students) == 0:
        break

# Cleanup
video_capture.release()
cv2.destroyAllWindows()
f.close()

print("Attendance session completed!")
print("Students marked present:", [name for name in known_faces_names if name not in students])

Attendance marked for: Dewmini (Live person verified)
Attendance marked for: Elon (Live person verified)
Attendance session completed!
Students marked present: ['Elon', 'Dewmini']
